In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [4]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4") 

prompt = ChatPromptTemplate.from_template("Write a short summary about {topic}.")


chain = prompt | llm | StrOutputParser()
result = chain.invoke({"topic": "the history of the internet"})
print(result)

The history of the internet dates back to the 1960s when the U.S. Department of Defense created ARPANET, a project that connected computers at various universities and research labs. In the 1970s, the concept of email was developed and the TCP/IP protocols, which serve as the backbone of the internet communication, were created. The 1980s saw the development of the Domain Name System (DNS) and the birth of the World Wide Web by Tim Berners-Lee in 1989. The 1990s marked the beginning of the internet's commercialization, with the launch of the first web browser, Mosaic, and the establishment of popular websites like Yahoo and Amazon. The 2000s brought about the rise of social media platforms, video streaming, and mobile internet. Today, the internet continues to evolve, with advancements in cloud computing, IoT, and AI shaping its future.


In [5]:
# modification 
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
llm = ChatOpenAI(temperature=0.0, model="gpt-4") 


In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])


chain = ({  
        
        "chat_history": lambda x: x.get("chat_history", []),
        "input": lambda x: x["input"]
}
| prompt | llm | StrOutputParser() 
)


chat_history =[
    ("system", "You are a helpful assistant."),
    ("human", "What is the capital of France?"),
    ("assistant", "The capital of France is Paris."),
    ("human", "What is the capital of Germany?"),
    ("assistant", "The capital of Germany is Berlin.")   
]

result = chain.invoke({"input": "What is the capital of India?", "chat_history": chat_history})


In [7]:
print(result)

The capital of India is New Delhi.


In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableLambda
llm = ChatOpenAI()

In [9]:
summary_chain=( ChatPromptTemplate.from_template("Write a short summary about {text}.") | llm | StrOutputParser()
)

analysis_chain = (ChatPromptTemplate.from_template("Analyze the summary: {text}.") | llm | StrOutputParser())

# parellel LCEL 
combined_chain = {
    "summary": summary_chain,
    "analysis": analysis_chain
} | RunnableLambda(
    lambda x: {
        "summary": x["summary"],
        "analysis": x["analysis"]
    }
)

comined_chain1 = combined_chain.invoke({"text": "the history of the internet"})

In [10]:
print(comined_chain1)

{'summary': 'The internet has a long and complex history that dates back to the 1960s. It began as a research project funded by the US government called ARPANET, which connected four computers at research institutions to share information and resources. Over the years, the internet evolved and expanded, with the development of protocols like TCP/IP that allowed for data exchange between different networks. The introduction of the World Wide Web in the 1990s made the internet accessible to the general public, leading to its widespread adoption and rapid growth. Today, the internet is an integral part of everyday life, connecting people and devices around the world and facilitating communication, commerce, and information exchange.', 'analysis': 'The summary likely covers the origins and development of the internet, including key milestones and advancements that have shaped its evolution. It may discuss how the internet was originally created for military and academic purposes before exp

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableMap,
    RunnableBranch,
    RunnableParallel
)

In [12]:
prompt = PromptTemplate.from_template("Write a short tweet about {topic}.")

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
parser = StrOutputParser()

In [14]:
add_suffix = RunnableLambda(lambda x: {"topic": x["topic"] + " for developers"})
passthrough = RunnablePassthrough()


In [15]:
multi_topic_map = RunnableMap({
    "ai": RunnableLambda(lambda _: {"topic": "AI"}),
    "ml": RunnableLambda(lambda _: {"topic": "ML"}),
    "cv": RunnableLambda(lambda _: {"topic": "Computer Vision"})
})


In [16]:
full_chain = (
    add_suffix |
    passthrough |
    prompt |
    llm |
    parser
)

In [17]:
full_chain.invoke({"topic": "Python"})

'"Python is a powerful and versatile language that makes development faster and easier. Perfect for both beginners and experienced developers! 🐍💻 #python #coding #development"'

In [18]:
print("The End")

The End
